In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


In [9]:
# Load the data
# Specify the file path
file_path = "./dataset/train.txt"

# Read the contents of the file
with open(file_path, 'r', encoding='utf-8') as file:
    data_content = file.read()

# Split the data into lines
X_train = data_content.split('\n')

def remove_non_arabic(text):
    # Use regex to keep only Arabic letters, spaces, tashkeel, and longation
    arabic_letters = re.sub(r'[^ ؀-ۿ\u0617-\u061A\u064B-\u0652\u0640]', '', text)
    # Remove tasheel
    arabic_letters = re.sub(r'[\u064B-\u0652]', '', arabic_letters)
    # Remove longation
    arabic_letters = re.sub(r'[\u0640]', '', arabic_letters)
    # Remove 3alamat tarkeem (.  ,  ~  :  !  ?  ;  ؟)
    arabic_letters = re.sub(r'[\.\,\~\:\!\؟\؛]', '', arabic_letters)
    # Remove extra spaces
    arabic_letters = re.sub(r'\s+', ' ', arabic_letters)
    
    return arabic_letters


# # Remove non-Arabic letters
# X_train = [remove_non_arabic(text) for text in X_train]



# print the first 2:6 lines in different lines
for i in range(2,6):
    print(X_train[i])



( قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ ) أَيْ الْوَصِيَّةُ ( قَوْلُهُ مَا مَرَّ ) أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي أَنَّهَا لَا تَبْطُلُ ، وَعِبَارَةُ الْكَنْزِ وَلَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِنْ غَنَمِي لَمْ يَتَعَيَّنْ غَنَمُهُ إنْ كَانَتْ انْتَهَتْ ا ه سم ( قَوْلُهُ فَيُعْطَى وَاحِدَةً مِنْهَا إلَخْ ) كَمَا لَوْ كَانَتْ مَوْجُودَةً عِنْدَ الْوَصِيَّةِ وَالْمَوْتِ ، وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ فِي الصُّورَتَيْنِ وَإِنْ تَرَاضَيَا ؛ لِأَنَّهُ صُلْحٌ عَلَى مَجْهُولٍ مُغْنِي وَنِهَايَةٌ قَالَ ع ش قَوْلُهُ وَاحِدَةً مِنْهَا أَيْ كَامِلَةً ، وَلَا يَجُوزُ أَنْ يُعْطَى نِصْفَيْنِ مِنْ شَاتَيْنِ ؛ لِأَنَّهُ لَا يُسَمَّى شَاةً وَقَوْلُهُ وَلَا يَجُوزُ أَنْ يُعْطَى وَاحِدَةً مِنْ غَيْرِ غَنَمِهِ وَيَنْبَغِي أَن

In [12]:
def extract_arabic_diacritics(text):
    """Extracts Arabic diacritics from the given text.

    Args:
        text: The Arabic text from which to extract diacritics.

    Returns:
        A list of the extracted diacritics.
    """

    diacritics_pattern = r"[\u064B-\u065F]"  # Matches all Arabic diacritics
    # Remove all non-Arabic letters
    text = re.sub(r'[\u064B-\u065F]', '', text)
    
    diacritics = re.findall(diacritics_pattern, text)
    return text,diacritics

extract_arabic_diacritics(X_train[2])

('( قوله لعدم ما تتعلق إلخ ) أي الوصية ( قوله ما مر ) أي قبيل قول المتن لغت ولو اقتصر على أوصيت له بشاة أو أعطوه شاة ولا غنم له عند الموت هل تبطل الوصية أو يشترى له شاة ويؤخذ من قوله الآتي كما لو لم يقل من مالي ولا من غنمي أنها لا تبطل ، وعبارة الكنز ولو لم يقل من مالي ولا من غنمي لم يتعين غنمه إن كانت انتهت ا ه سم ( قوله فيعطى واحدة منها إلخ ) كما لو كانت موجودة عند الوصية والموت ، ولا يجوز أن يعطى واحدة من غير غنمه في الصورتين وإن تراضيا ؛ لأنه صلح على مجهول مغني ونهاية قال ع ش قوله واحدة منها أي كاملة ، ولا يجوز أن يعطى نصفين من شاتين ؛ لأنه لا يسمى شاة وقوله ولا يجوز أن يعطى واحدة من غير غنمه وينبغي أن يقال مثل ذلك في الأرقاء ا ه .',
 [])